<a href="https://colab.research.google.com/github/CheayeonLee/24_2_MainSession/blob/master/%ED%8B%B1%ED%83%9D%ED%86%A0_State.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# State class
틱택토 상태를 나타낼 수 있는 class를 만들어 주세요.

- 2명의 플레이어가 있고, 각 플레이어의 수가 뭔지 식별가능해야 합니다.  

**꼭 구현해야할 매소드(주니어/시니어 공통)**
- 둘 수 있는 위치가 어딘지 알려주는 함수
- win / lose / draw를 식별해주는 함수
- render 함수

**min-max algorithm / alpha-beta algorithm 구현(시니어)**

### 🚨 만약 감이 안 온다면 운영진에게 연락주시면 참고할 만한 레퍼런스를 보내드리겠습니다.
https://github.com/Jpub/AlphaZero/blob/master/6_7_tictactoe/game.py
### 🚨 참고한 레퍼런스가 있다면, 출처를 남겨주세요.

https://symxsym.tistory.com/entry/pygame-%ED%8C%8C%EC%9D%B4%EC%8D%AC%EC%9C%BC%EB%A1%9C-%EA%B2%8C%EC%9E%84-%EB%A7%8C%EB%93%A4%EA%B8%B0-%ED%8B%B1%ED%83%9D%ED%86%A0

https://github.com/kekmodel/gym-tictactoe-zero/blob/master/selfplay_gpu.py render있고 교재에서 나온 코드임

https://medium.com/byte-sized-code/building-a-game-of-tic-tac-toe-in-python-72dd59c5f240
pygame 안 쓰는거

https://m.blog.naver.com/lsszz210/222238307295

In [ ]:
# 패키지 임포트
import random
import math


class State:
    # 초기화
    def __init__(self, pieces=None, enemy_pieces=None):
        # 돌의 배치
        self.pieces = pieces if pieces != None else [0] * 9
        self.enemy_pieces = enemy_pieces if enemy_pieces != None else [0] * 9

    # 돌의 수 얻기
    def piece_count(self, pieces):
        count = 0
        for i in pieces:
            if i == 1:
                count += 1
        return count

    # 승리 여부 판정
    def is_win(self):
        # 돌 3개 연결 여부 확인
        def is_comp(x, y, dx, dy):
            for k in range(3):
                if y < 0 or 2 < y or x < 0 or 2 < x or \
                        self.pieces[x + y * 3] == 0:
                    return False
                x, y = x + dx, y + dy
            return True

        # 승리 여부 판정
        if is_comp(0, 0, 1, 1) or is_comp(0, 2, 1, -1):
            return True
        for i in range(3):
            if is_comp(0, i, 1, 0) or is_comp(i, 0, 0, 1):
                return True
        return False

    # 패배 여부 판정
    def is_lose(self):
        # 돌 3개 연결 여부 확인
        def is_comp(x, y, dx, dy):
            for k in range(3):
                if y < 0 or 2 < y or x < 0 or 2 < x or \
                        self.enemy_pieces[x + y * 3] == 0:
                    return False
                x, y = x + dx, y + dy
            return True

        # 패배 여부 판정
        if is_comp(0, 0, 1, 1) or is_comp(0, 2, 1, -1):
            return True
        for i in range(3):
            if is_comp(0, i, 1, 0) or is_comp(i, 0, 0, 1):
                return True
        return False

    # 무승부 여부 확인
    def is_draw(self):
        return self.piece_count(self.pieces) + self.piece_count(self.enemy_pieces) == 9

    # 게임 종료 여부 확인
    def is_done(self):
        return self.is_win() or self.is_lose() or self.is_draw()

    # 다음 상태 얻기
    def next(self, action):
        pieces = self.pieces.copy()
        pieces[action] = 1
        return State(self.enemy_pieces, pieces)

    # 합법적인 수의 리스트 얻기
    def legal_actions(self):
        actions = []
        for i in range(9):
            if self.pieces[i] == 0 and self.enemy_pieces[i] == 0:
                actions.append(i)
        return actions

    # 선 수 여부 확인
    def is_first_player(self):
        return self.piece_count(self.pieces) == self.piece_count(self.enemy_pieces)

    # # 문자열 표시
    # def __str__(self):
    #     ox = ('o', 'x') if self.is_first_player() else ('x', 'o')
    #     str = ''
    #     for i in range(9):
    #         if self.pieces[i] == 1:
    #             str += ox[0]
    #         elif self.enemy_pieces[i] == 1:
    #             str += ox[1]
    #         else:
    #             str += '-'
    #         if i % 3 == 2:
    #             str += '\n'
    #     return str
    def render(self):
        ox = ('o', 'x') if self.is_first_player() else ('x', 'o')
        for i in range(9):
            if self.pieces[i] == 1:
                print(ox[0], end='')  # 자신의 돌 표시
            elif self.enemy_pieces[i] == 1:
                print(ox[1], end='')  # 상대의 돌 표시
            else:
                print('-', end='')  # 빈 칸 표시

            if i % 3 == 2:  # 3번째 돌마다 줄바꿈
                print()

In [ ]:
pieces = [1, 0, 0,
               0, 1, 0,
               0, 0, 0]

enemy_pieces = [0, 1, 0,
                     0, 0, 1,
                     0, 0, 0]
st = State(pieces, enemy_pieces)
st.render()

ox-
-ox
---


In [ ]:
# 랜덤으로 행동 선택
def random_action(state):
    legal_actions = state.legal_actions()
    return legal_actions[random.randint(0, len(legal_actions) - 1)]


# 알파베타법을 활용한 상태 가치 계산
def alpha_beta(state, alpha, beta):
    # 패배 시 상태 가치 -1
    if state.is_lose():
        return -1

    # 무승부 시, 상테 가치 0
    if state.is_draw():
        return 0

    # 합법적인 수의 상태 가치 계산
    for action in state.legal_actions():
        score = -alpha_beta(state.next(action), -beta, -alpha)
        if score > alpha:
            alpha = score

        # 현재 노드의 베스트 스코어가 부모 노드보다 크면 탐색 종료
        if alpha >= beta:
            return alpha

    # 합법적인 수의 강태 가치의 최대값을 반환
    return alpha


# 알파베타법을 활용한 생동 선택
def alpha_beta_action(state):
    # 합법적인 수의 상태 가치 계산
    best_action = 0
    alpha = -float('inf')
    for action in state.legal_actions():
        score = -alpha_beta(state.next(action), -float('inf'), -alpha)
        if score > alpha:
            best_action = action
            alpha = score

    # 합법적인 수의 상태 가치의 최대값을 갖는 행동 반환
    return best_action


# 플레이아웃
def playout(state):
    # 패배 시, 상태 가치 -1
    if state.is_lose():
        return -1

    # 무승부 시, 상태 가치 0
    if state.is_draw():
        return 0

    # 다음 상태의 상태 가치
    return -playout(state.next(random_action(state)))


# 최대값의 인덱스 반환
def argmax(collection):
    return collection.index(max(collection))


# 몬테카를로 트리 탐색을 활용한 행동 선택
def mcts_action(state):
    # 몬테카를로 트리 탐색 노드
    class node:
        # 초기화
        def __init__(self, state):
            self.state = state  # 상태
            self.w = 0  # 가치 누계
            self.n = 0  # 시행 횟수
            self.child_nodes = None  # 자녀 노드군

        # 평가
        def evaluate(self):
            # 게임 종료 시
            if self.state.is_done():
                # 승패 결과로 가치 얻기
                value = -1 if self.state.is_lose() else 0  # 패배 시 -1, 무승부 시 0

                # 가치 누계와 시행 횟수 갱신
                self.w += value
                self.n += 1
                return value

            # 자녀 노드가 존재하지 않는 경우
            if not self.child_nodes:
                # 플레이아웃으로 가치 얻기
                value = playout(self.state)

                # 가치 누계와 시행 횟수 갱신
                self.w += value
                self.n += 1

                # 자녀 노드 전개
                if self.n == 10:
                    self.expand()
                return value

            # 자녀 노드가 존재하는 경우
            else:
                # UCB1이 가장 큰 자녀 노드를 평가해 가치 얻기
                value = -self.next_child_node().evaluate()

                # 보상 누계와 시행 횟수 갱신
                self.w += value
                self.n += 1
                return value

        # 자녀 노드 전개
        def expand(self):
            legal_actions = self.state.legal_actions()
            self.child_nodes = []
            for action in legal_actions:
                self.child_nodes.append(node(self.state.next(action)))

        # UCB1가 가장 큰 자녀 노드 얻기
        def next_child_node(self):
            # 시행 횟수 n이 0인 자녀 노드를 반환
            for child_node in self.child_nodes:
                if child_node.n == 0:
                    return child_node

            # UCB1 계산
            t = 0
            for c in self.child_nodes:
                t += c.n
            ucb1_values = []
            for child_node in self.child_nodes:
                ucb1_values.append(-child_node.w / child_node.n + 2 * (2 * math.log(t) / child_node.n) ** 0.5)

            # UCB1가 가장 큰 자녀 노드를 반환
            return self.child_nodes[argmax(ucb1_values)]

    # 루트 노드 생성
    root_node = node(state)
    root_node.expand()

    # 루트 노드를 100회 평가
    for _ in range(100):
        root_node.evaluate()

    # 시행 횟수 최대값을 갖는 행동 반환
    legal_actions = state.legal_actions()
    n_list = []
    for c in root_node.child_nodes:
        n_list.append(c.n)
    return legal_actions[argmax(n_list)]


# 동작 확인
if __name__ == '__main__':
    # 상태 생성
    state = State()

    # 게임 종료 시까지 반복
    while True:
        # 게임 종료 시
        if state.is_done():
            break

        # 다음 상태 얻기
        state = state.next(random_action(state))

        # 문자열 표시
        print(state)
        print()